In [1]:
from datasets import load_dataset

dataset = load_dataset("embedding-data/sentence-compression")

/opt/anaconda3/envs/wl-upgrade/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 277.40it/s]


Dataset json downloaded and prepared to /Users/tollef/.cache/huggingface/datasets/embedding-data___json/embedding-data--sentence-compression-d643585deb6e0073/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 195.50it/s]


In [2]:
dataset["train"][18000]

{'set': ["Dubai The Dubai developer Majid Al Futtaim is planning a major new mall in Abu Dhabi which could be as big as Dubai's Mirdiff City Centre.",
  'Majid Al Futtaim plans a major new mall in Abu Dhabi']}

In [3]:
wikianswers25 = load_dataset("embedding-data/WikiAnswers")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 325.52it/s]


KeyboardInterrupt: 

In [4]:
wikianswers25["train"]

Dataset({
    features: ['set'],
    num_rows: 27383151
})

In [84]:
# select a subset of 10000 examples of 25 sentences:

wikisubset = wikianswers25["train"].shuffle(seed=42).select(range(100000))

Loading cached shuffled indices for dataset at /Users/tollef/.cache/huggingface/datasets/embedding-data___json/embedding-data--WikiAnswers-6933ed83395546bb/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-d44a4d6c10461af3.arrow


In [90]:
import jsonlines
import rich
import os

def process(wiki_example):
    wiki_example.sort(key=len)
    # select the idx 2, as the absolute shortest usually seems wrong.
    n = len(wiki_example)
    # return three pairs, one LONGEST->AVG, AVG->SHORTEST, LONGEST->SHORTEST
    return [
        [wiki_example[-1], wiki_example[n//2]],
        [wiki_example[n//2], wiki_example[0]],
        [wiki_example[-1], wiki_example[0]]
    ]

filename = "wikianswers_3length-100000samples.jsonl"
filepath = "../../data/" + filename
if os.path.exists(filepath):
    os.remove(filepath)

with jsonlines.open(filepath, mode="a") as writer:
    for i, example in enumerate(wikisubset):
        processed = process(example["set"])
        # remove duplicates
        processed = list(set([tuple(p) for p in processed]))
        for p in processed:
            writer.write({"set": p})